In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Chicken-Disease-Classifier-With-Mlflow-and-DVC'

In [ ]:
# create config entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_augumentation:bool
    params_image_size: list

In [7]:
# update configuration manager

from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
# add configureation mananger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,   # from constants
        params_filepath=PARAMS_FILE_PATH    # notice the S
    ):
        # Load YAMLs
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Make artifacts directory if missing
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_augumentation=params.AUGUMENTATION,
            params_image_size=params.IMAGE_SIZE
        ) 

        return training_config

In [10]:
# update component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [45]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            # validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            # directory=self.config.training_data,
            directory=os.path.join(self.config.training_data, "valid"),
            # subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_augumentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                # **dataflow_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            # directory=self.config.training_data,
            directory=os.path.join(self.config.training_data, "train"),
            # subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        

        # self.valid_generator = train_datagenerator.flow_from_directory(
            # directory=self.config.training_data,
            # directory=os.path.join(self.config.training_data, "train")
            # subset="training",
            # shuffle=False,
            # **dataflow_kwargs
        # )   

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [46]:
try:
    config=ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2026-01-01 16:46:28,065]: INFO: common: yaml file: config\config.yaml loaded successfully:
[2026-01-01 16:46:28,070]: INFO: common: yaml file: params.yaml loaded successfully:
[2026-01-01 16:46:28,075]: INFO: common: created directory at: artifacts:
[2026-01-01 16:46:28,078]: INFO: common: created directory at: artifacts\training:
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
38/38 [==============================] - 40s 929ms/step - loss: 73.0407 - accuracy: 0.3752 - val_loss: 12.0653 - val_accuracy: 0.3125


In [35]:
print("Training directory:", self.train_dir)


NameError: name 'self' is not defined